In [ ]:
!pip install sodapy # To work with data
!pip install folium # To plot data

In [ ]:
from pandas.core import api
import datetime
from sodapy import Socrata
import pandas as pd
import folium
import numpy as np
import seaborn as sns

In [ ]:
client = Socrata("data.cityofnewyork.us", None)

In [ ]:
# Define the current date and one year ago
today = datetime.date.today()
one_year_ago = today - datetime.timedelta(days=365)

# Set the query parameters
query_params = (
    f"borough='MANHATTAN' AND "
    f"(number_of_persons_injured>0 OR number_of_persons_killed>0) AND "
    f"crash_date>='{one_year_ago}'"
)

# Query the data
results = client.get("h9gi-nx95", where=query_params, limit=10000)

df = pd.DataFrame.from_records(results)
# Infer data types
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [ ]:
df = df.dropna(subset=["latitude", "longitude"])
df = df[(df['latitude'] > 40.494101) & (df['latitude'] < 40.871775)]

In [ ]:
query_params = "factype='AMBULANCE STATION' AND boro='MANHATTAN'"

# Query the data
ambulance_stations = client.get("ji82-xba5", where=query_params)

# Convert the results into a DataFrame
ambulance_stations_df = pd.DataFrame.from_records(ambulance_stations)
ambulance_stations_df['latitude'] = ambulance_stations_df['latitude'].astype(float)
ambulance_stations_df['longitude'] = ambulance_stations_df['longitude'].astype(float)

In [ ]:

# Set the query parameters as a string
query_params = "factype='HOSPITAL' AND boro='MANHATTAN'"

# Query the data
hospitals = client.get("ji82-xba5", where=query_params)

# Convert the results into a DataFrame
hospitals_df = pd.DataFrame.from_records(hospitals)

hospitals_df['latitude'] = hospitals_df['latitude'].astype(float)
hospitals_df['longitude'] = hospitals_df['longitude'].astype(float)


In [ ]:

# Function to create a popup with details for each accident
def create_accident_popup(row):
    return folium.Popup(
        f"Date: {row['crash_date']}<br>"
        f"Time: {row['crash_time']}<br>"
        f"Injured: {row['number_of_persons_injured']}<br>"
        f"Killed: {row['number_of_persons_killed']}<br>"
    )

# Function to create a popup with details for each ambulance station
def create_ambulance_station_popup(row):
    return folium.Popup(
        f"Name: {row['facname']}<br>"
    )

# Function to create a popup with details for each hospital
def create_hospital_popup(row):
    return folium.Popup(
        f"Name: {row['facname']}<br>"
    )


In [ ]:


# Create a map centered on Manhattan
nyc_map = folium.Map(location=[40.7831, -73.9712], zoom_start=12)

# Create feature groups for accidents, ambulance stations, and hospitals
accidents_group = folium.FeatureGroup(name="Accidents")
ambulance_stations_group = folium.FeatureGroup(name="Ambulance Stations")
hospitals_group = folium.FeatureGroup(name="Hospitals")

# Add a circle for each accident to the accidents feature group
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=4,
        popup=create_accident_popup(row),
        color="red" if int(row["number_of_persons_killed"]) > 0 else "blue",
        fill=True,
        fill_color="red" if int(row["number_of_persons_killed"]) > 0 else "blue",
        fill_opacity=0.7,
    ).add_to(accidents_group)

# Add a circle for each ambulance station to the ambulance stations feature group
for index, row in ambulance_stations_df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=6,
        popup=create_ambulance_station_popup(row),
        color="green",
        fill=True,
        fill_color="green",
        fill_opacity=0.7,
    ).add_to(ambulance_stations_group)

# Add a circle for each hospital to the hospitals feature group
for index, row in hospitals_df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=10,
        popup=create_hospital_popup(row),
        color="purple",
        fill=True,
        fill_color="purple",
        fill_opacity=0.7,
    ).add_to(hospitals_group)

# Add the feature groups to the map
accidents_group.add_to(nyc_map)
ambulance_stations_group.add_to(nyc_map)
hospitals_group.add_to(nyc_map)

# Add a layer control to the map
folium.LayerControl().add_to(nyc_map)




In [ ]:
# Display the map
nyc_map

In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitudes and longitudes from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula 
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))

    # Earth's radius in kilometers (mean radius = 6,371 km)
    radius = 6371

    # Calculate the distance
    distance = c * radius

    return distance


In [ ]:
combined_facilities_df = pd.concat([ambulance_stations_df, hospitals_df], ignore_index=True)

In [ ]:
df["closest_facility"] = np.nan
df["closest_facility_distance"] = np.nan

In [ ]:
for index, row in df.iterrows():
    min_distance = 1000000
    closest_facility = None
    for facility_index, facility_row in combined_facilities_df.iterrows():
        distance = haversine(row["latitude"], row["longitude"], facility_row["latitude"], facility_row["longitude"])
        if distance < min_distance:
            min_distance = distance
            closest_facility = facility_row["facname"]
    df.loc[index, "closest_facility"] = closest_facility
    df.loc[index, "distance_to_closest_facility"] = min_distance

In [ ]:
important_columns = ["crash_date","crash_time","borough","closest_facility","distance_to_closest_facility","number_of_persons_injured","number_of_persons_killed","latitude","longitude"]

In [ ]:
df[important_columns]

In [ ]:
def display_config(df, facilities_df, new_facility_latitude=None, new_facility_longitude=None):
    nyc_map = folium.Map(location=[40.7831, -73.9712], zoom_start=12)


    accidents_group = folium.FeatureGroup(name="Accidents")
    facilities_group = folium.FeatureGroup(name="Hospitals")
    new_facility_group = folium.FeatureGroup(name="New Hospital")

    # Get unique hospital names
    unique_hospitals = df["closest_facility"].unique()

    # Get a list of colors (use the seaborn library for generating colors)
    colors = sns.color_palette("hls", len(unique_hospitals))
    hex_colors = colors.as_hex()

    # Create a dictionary mapping hospital names to colors
    facility_colors = dict(zip(unique_hospitals, hex_colors))

    # Add a circle for each ambulance station to the ambulance stations feature group
    for index, row in facilities_df.iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            radius=20,
            popup=create_ambulance_station_popup(row),
            icon=folium.Icon(color="green"),
        ).add_to(facilities_group)

     # Iterate through the DataFrame rows and add a circle for each row
    for _, row in df.iterrows():
        lat, lon = row["latitude"], row["longitude"]
        facility = row["closest_facility"]
        color = facility_colors[facility]

        folium.Circle(
            location=[lat, lon],
            radius=40,  # Set the circle size (in meters)
            popup=facility,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1
        ).add_to(nyc_map)

    # Add a circle for the new facility
    if new_facility_latitude and new_facility_longitude:
        folium.Marker(
            location=[new_facility_latitude, new_facility_longitude],
            radius=15,
            popup="NEW HOSPITAL",
            icon=folium.Icon(color="red"),
        ).add_to(new_facility_group)

    # Add the feature groups to the map
    accidents_group.add_to(nyc_map)
    facilities_group.add_to(nyc_map)
    new_facility_group.add_to(nyc_map)

    # Add a layer control to the map
    folium.LayerControl().add_to(nyc_map)
    
    return nyc_map

In [ ]:


def reassign_closest_facility(df, new_lat, new_lon):
    df_ = df.copy()
    for index,row in df_.iterrows():
        new_distance = haversine(row["latitude"], row["longitude"], new_lat, new_lon)
        old_distance = row['distance_to_closest_facility']
        if new_distance < old_distance:
            df_.loc[index,'distance_to_closest_facility'] = new_distance
            df_.loc[index,'closest_facility']='NEW'
    return df_

    

In [ ]:
display_config(df,combined_facilities_df)

In [ ]:
# new_lat,new_lon = combined_facilities_df[['latitude','longitude']].iloc[0]
new_lat,new_lon = 40.7831, -73.9712
new_config = reassign_closest_facility(df,new_lat,new_lon)

In [ ]:
display_config(new_config,combined_facilities_df,new_lat,new_lon)

In [ ]:

def get_points_x_meters_away(latitude, longitude, distance_meters):
    meters_per_degree_latitude = 111111
    meters_per_degree_longitude = meters_per_degree_latitude * math.cos(math.radians(latitude))

    lat_change = distance_meters / meters_per_degree_latitude
    lon_change = distance_meters / meters_per_degree_longitude

    north = (latitude + lat_change, longitude)
    south = (latitude - lat_change, longitude)
    east = (latitude, longitude + lon_change)
    west = (latitude, longitude - lon_change)

    return north, south, east, west

In [ ]:
def best_new_facility(df, combined_facilities_df):
    best_cost=1e10
    best_lat,best_lon = None,None
    for i in range(len(combined_facilities_df)):
        current_lat,current_lon = combined_facilities_df[['latitude','longitude']].iloc[i]
        cost, s = add_new_facility(df, current_lat,current_lon)
        print("Cost for",s,"is",cost)
        if cost < best_cost:
            best_cost = cost
            best_lat,best_lon = s
    print("BEST one is", best_cost, best_lat,best_lon)
    return best_lat,best_lon
        


def add_new_facility(df, current_lat,current_lon):
    current_config = reassign_closest_facility(df,current_lat,current_lon)
    current_config_cost = current_config['distance_to_closest_facility'].sum()
    distance=1000

    while True:
        # Check cost on all 4 sides
        print(current_lat,current_lon, current_config_cost)
        costs = []
        for s in get_points_x_meters_away(current_lat,current_lon,distance):
            new_config = reassign_closest_facility(df,s[0],s[1])
            new_config_cost = new_config['distance_to_closest_facility'].sum()
        
            costs.append([new_config_cost, s])
        
        # Get the best configuration
        best = min(costs)
        best_cost = best[0]
        best_lat,best_lon = best[1]

        if best_cost < current_config_cost:
            current_config_cost = best_cost
            current_lat,current_lon = best_lat,best_lon
        else:
            break

    return current_config_cost, (current_lat,current_lon)


In [ ]:
new_lat, new_lon = best_new_facility(df, combined_facilities_df)
current_config = reassign_closest_facility(df,new_lat, new_lon)

In [ ]:

display_config(current_config,combined_facilities_df,new_lat, new_lon)